## Cogreg Using Rhino - Oxford Pilot Data 20241216

In [1]:
# We start by importing the relevant packages
import osl_ephys
import numpy as np
import mne
import glob
import yaml
import os
import matplotlib.pyplot as plt

# Set global font to Open Sans
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica']
plt.rcParams['font.size'] = 12  # Adjust font size as desired


## Get FSL Setup

In [2]:
from osl_ephys import source_recon
fsl_dir = '/Users/robertseymour/fsl'
source_recon.setup_fsl(fsl_dir)

## Specify Subject

In [3]:
subject = 'seamus'

## Load in the Data - the downsampled headshape version

In [4]:
filename = 'BIDS/sub-{}/ses-001/meg/sub-{}_ses-001_task-fourMotor_run-001_meg_hs_downsample-raw.fif'.format(subject,subject)
raw = mne.io.read_raw_fif(filename, preload=True)

Opening raw data file BIDS/sub-seamus/ses-001/meg/sub-seamus_ses-001_task-fourMotor_run-001_meg_hs_downsample-raw.fif...
    Range : 0 ... 2198783 =      0.000 ...  1465.855 secs
Ready.
Reading 0 ... 2198783  =      0.000 ...  1465.855 secs...


In [5]:
# Plot the sensors to check they are in the right orientation
fig = mne.viz.plot_alignment(raw.info,dig=True)

Using pyvistaqt 3d backend.
Getting helmet for system unknown (derived from 192 MEG channel locations)
Channel types::	mag: 192


## Setup subjects_dir

In [5]:
subjects_dir = ''


## Run MRI through FSL

In [ ]:
source_recon.rhino.compute_surfaces(
    'SOS.nii',
    subjects_dir=subjects_dir,
    subject=subject,
    include_nose=True,
    do_mri2mniaxes_xform=True
)

In [6]:
source_recon.rhino.surfaces.surfaces_display("", "seamus")

### Save the headshape data to the right place

In [9]:
from osl_ephys.source_recon.rhino.utils import save_polhemus_fif
save_polhemus_fif(raw,subjects_dir=subjects_dir,subject=subject)

Saved files to seamus/rhino/coreg


### Do Coreg!

In [9]:
source_recon.rhino.coreg(
    filename,
    subjects_dir=subjects_dir,
    subject=subject,
    use_headshape=True,
    use_nose=True,
    n_init = 10,
)

*** RUNNING OSL-EPHYS RHINO COREGISTRATION ***
Opening raw data file BIDS/sub-seamus/ses-001/meg/sub-seamus_ses-001_task-fourMotor_run-001_meg_hs_downsample-raw.fif...
    Range : 0 ... 2198783 =      0.000 ...  1465.855 secs
Ready.
Creating RawArray with float64 data, n_channels=218, n_times=1
    Range : 0 ... 0 =      0.000 ...     0.000 secs
Ready.
Overwriting existing file.
Writing /Users/robertseymour/data/20241216_oxford_pilot/seamus/rhino/coreg/info-raw.fif
Closing /Users/robertseymour/data/20241216_oxford_pilot/seamus/rhino/coreg/info-raw.fif
[done]
The MRI-derived nose is going to be used to aid coreg.
Please ensure that rhino.compute_surfaces was run with include_nose=True.
Please ensure that the polhemus headshape points include the nose.
loading: seamus/rhino/coreg/polhemus_headshape.txt
loading: seamus/rhino/coreg/polhemus_nasion.txt
loading: seamus/rhino/coreg/polhemus_rpa.txt
loading: seamus/rhino/coreg/polhemus_lpa.txt
Using known MNI fiducials
Running ICP...
ICP found

### Display Headmodel

In [10]:
source_recon.rhino.bem_display(
    subjects_dir=subjects_dir,
    subject=subject,
    display_outskin_with_nose=False,
    display_sensors=True,
    plot_type="surf",

)

Reading forward solution from /Users/robertseymour/data/20241216_oxford_pilot/seamus/rhino/model-fwd.fif...
    Reading a source space...
    [done]
    1 source spaces read
    Desired named matrix (kind = 3523 (FIFF_MNE_FORWARD_SOLUTION_GRAD)) not available
    Read MEG forward solution (2863 sources, 192 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
BEM surface: number of dipoles = 2863
Overwriting existing file.


## Display Coreg Accuracy

#### Note: requires the sensor type to be added to _3d.py MNE code - I need to make a pull request to MNE 

In [6]:
source_recon.rhino.coreg_display(
    subjects_dir=subjects_dir,
    subject=subject,
    display_outskin_with_nose=True,
    display_sensors=True,
    display_sensor_oris = True,
    plot_type="surf",
    display_headshape_pnts=True,
    display_outskin = True,
    # filename='./bem_dispay.html',

)

Using pyvistaqt 3d backend.
Creating surface mesh for seamus/rhino/coreg/scaled_outskin_plus_nose_mesh.nii.gz .....
Overwriting existing file.


### Create 8mm grid

In [7]:
gridstep = 8
source_recon.rhino.forward_model(
    subjects_dir=subjects_dir,
    subject=subject,
    model="Single Layer",
    gridstep=gridstep,
)

*** RUNNING OSL-EPHYS RHINO FORWARD MODEL ***
Overwriting existing file.
Using bet_inskull_surf_file for single shell surface
Overwriting existing file.
Overwriting existing file.
Surface CM = (  -1.6   -6.3   12.0) mm
Surface fits inside a sphere with radius   96.8 mm
Surface extent:
    x =  -75.4 ...   74.2 mm
    y =  -83.1 ...   86.4 mm
    z =  -65.0 ...   83.2 mm
Grid extent:
    x =  -80.0 ...   80.0 mm
    y =  -88.0 ...   88.0 mm
    z =  -72.0 ...   88.0 mm
10143 sources before omitting any.
6963 sources after omitting infeasible sources not within 0.0 - 96.8 mm.
Source spaces are in MRI coordinates.
Checking that the sources are inside the surface and at least    4.0 mm away (will take a few...)
Checking surface interior status for 6963 points...
    Found  365/6963 points inside  an interior sphere of radius   35.6 mm
    Found    0/6963 points outside an exterior sphere of radius   96.8 mm
    Found 3352/6598 points outside using surface Qhull
    Found  268/3246 points o

### Read Forward Solution and Plot

In [8]:
from mne import read_forward_solution

# load forward solution
fwd_fname = '{}/rhino/model-fwd.fif'.format(subject)
fwd = read_forward_solution(fwd_fname)

Reading forward solution from /Users/robertseymour/data/20241216_oxford_pilot/seamus/rhino/model-fwd.fif...
    Reading a source space...
    [done]
    1 source spaces read
    Desired named matrix (kind = 3523 (FIFF_MNE_FORWARD_SOLUTION_GRAD)) not available
    Read MEG forward solution (2863 sources, 192 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame


In [11]:
# Visualize the forward solution using plot_alignment() (corrected)
mne.viz.plot_alignment(
    raw.info,  # Use the 'info' from the forward solution
    dig=True,
    subject=subject,  # Name of the subject
    subjects_dir=subjects_dir,  # Path to the subject's MRI directory
    src=fwd['src'],
)

Getting helmet for system unknown (derived from 192 MEG channel locations)
Channel types::	mag: 192
